In [49]:
import psycopg2
import geopandas as gpd
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from config import config

# CONEXIÓN a la BASE DE DATOS

params = config()
conn = psycopg2.connect(**params)

comarca = 'vera'
zona = 'losar'

# CONDICIONES SINÓPTICAS

sql = '''WITH inc as 
        (SELECT a.*, b.descripcion_corta as causa
        FROM 
            incendios_proximos_ze a,
            causas b
        WHERE 
            a.idcausa = b.id
            and b.ididioma = 0
        ORDER by tot desc)
        SELECT inc.*, c.texto, c.descripcion
        FROM 
            inc,
            texto_situacion_sinoptica c
        WHERE
            c.idpif = inc.idpif
        '''

incendios_cs = gpd.read_postgis(sql, conn)
incendios_cs['epa'] = (pd.DatetimeIndex(incendios_cs.deteccion).month > 5) & (pd.DatetimeIndex(incendios_cs.deteccion).month < 11)
ss = incendios_cs[['idpif', 'descripcion']].groupby(['descripcion']).count()
ss.columns = ['Número de incendios']
ss.index.name = 'Situación sinóptica'
display(len(incendios_cs))
# CAUSAS

causas = pd.read_sql('''
            with c as 
                (select idcausa / 100 as ppal, count(*) as num, 
                sum(superficiearboladatotal) as sup_arbolada, 
                sum(superficinoarboladatotal) as sup_no_arbolada 
                from incendios_proximos_ze 
                group by idcausa 
                order by idcausa) 
            select 
                ppal, 
                sum(num) as num_incendios, 
                sum(sup_arbolada) as sup_arbolada, 
                sum(sup_no_arbolada) as sup_no_arbolada,
                sum(sup_arbolada) + sum(sup_no_arbolada) as sup_total
            from 
                c
            group by ppal 
            order by ppal
            ''', conn)

causas_detalle = pd.read_sql('''
            with c as 
                (select idcausa, count(*) as num, 
                sum(superficiearboladatotal) as sup_arbolada, 
                sum(superficinoarboladatotal) as sup_no_arbolada 
                from incendios_proximos_ze 
                group by idcausa 
                order by idcausa) 
            select 
                idcausa,
                descripcion_corta,
                sum(num) as num_incendios, 
                sum(sup_arbolada) as sup_arbolada, 
                sum(sup_no_arbolada) as sup_no_arbolada,
                sum(sup_arbolada) + sum(sup_no_arbolada) as sup_total
            from 
                c,
                causas 
            where 
                c.idcausa = causas.id
                and causas.ididioma = 0
            group by 
                descripcion_corta,
                idcausa
            order by c.idcausa
            ''', conn)
    
causas.index = ['Rayo', 'Negligencia', 'Accidente', 'Intencionado', 'Desconocida', 'Reproducción']
causas = causas[['num_incendios', 'sup_arbolada', 'sup_no_arbolada', 'sup_total']]
display(causas_detalle)
display(ss)

# SEVERIDAD 

incendios = [{'zona': 'Jerte-Tornavacas', 'idpif': ['1133522', '1143015', '1143273', '574176', '1164099']}, 
             {'zona': 'Villanueva-Madrigal', 'idpif': ['1122656', '1132420', '1133464', '1163320', '1163391']}, 
             {'zona': 'Losar', 'idpif': ['1163407', '569396', '1169767', '574250', '1122672']}]

severidad = []

zonas_dict = {'jerte': 0, 'tornavacas': 0, 'madrigal': 1, 'losar': 2}

severidad = incendios[zonas_dict[zona]]
severidad['incendios'] = []

for n in severidad['idpif']:
    perimetro = gpd.read_postgis("select * from perimetros_utm where idpif = {}".format(n), conn)
    display(perimetro)
    severidad['incendios'].append({'fecha': perimetro.iloc[0]['fecha'].strftime('%d/%m/%Y'), 'idpif': n})
    display(severidad)
# CATASTRO 

catastro = []
sql = '''
    SELECT a.*
    FROM catastro_{} a, zona_estudio_{} b
    WHERE st_intersects(a.geom, b.geom) 
    AND a.tipo = 'R'
    AND st_area(st_intersection(a.geom, b.geom)) > 50000
'''.format(zona, zona)

parcelas = gpd.read_postgis(sql, conn)
for key, parcela in parcelas.iterrows():
    catastro.append({'key': key,
                     'refcat': parcela['refcat'], 
                     'fecha': parcela['fechaalta'], 
#                      'propietario1': parcela['propietario1'], 
#                      'propietario2': parcela['propietario2'], 
#                      'contacto1': parcela['contacto1'], 
#                      'contacto2': parcela['contacto2'], 
#                      'nombre': parcela['nombre'], 
                     'sup': "{:.2f}".format(parcela['area'] / 10000.0)})

# FORESTALES
    
forestales = pd.read_sql("select forestal, contacto from forestales where zona = '{}'".format(zona), conn)
display(forestales)

# HIDROGRAFÍA

rios = gpd.read_postgis('''select 
                row_number() over (order by a.long_km desc) as id_mapa, a.* 
                from rios a, zona_estudio_{} b 
                where st_intersects(a.geom, b.geom)
                and a.long_km < 50'''.format(zona), conn)

# METEO (TODO crear tabla en base de datos para no tener tanto código aquí)
datos_meteo = []
for estacion in ['madrigal', 'piornal']:
    dias_lluvia = pd.read_sql('''
                            select estacion, count(*) as dias_lluvia
                            from meteo_{} 
                            where prectotal > 0 and estacion is not null
                            and estacion != 'invierno_2020_2021'
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    dias_sin_lluvia_5 = pd.read_sql('''
                            select estacion, count(*) as dsl5
                            from meteo_{} 
                            where dias_sin_lluvia_5 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    dias_sin_lluvia_8 = pd.read_sql('''
                            select estacion, count(*) as dsl8
                            from meteo_{} 
                            where dias_sin_lluvia_8 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    nieve = pd.read_sql('''
                            select estacion, count(*) as nieve
                            from meteo_{}
                            where prectotal > 0 and tmin < 0 and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    heladas = pd.read_sql('''
                            select estacion, count(*) as heladas
                            from meteo_{}
                            where tmin < 0 and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    ventana_solana = pd.read_sql('''
                            select estacion, count(*) as vs
                            from meteo_{} 
                            where tmax < 21 and tmin > 6 and dias_sin_lluvia_5 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    ventana_umbria = pd.read_sql('''
                            select estacion, count(*) as vu
                            from meteo_{}
                            where tmax < 21 and tmin > 6 and dias_sin_lluvia_8 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)

    meteo = pd.merge(dias_lluvia, dias_sin_lluvia_5, how='left')
    meteo = pd.merge(meteo, dias_sin_lluvia_8, how='left')
    meteo = pd.merge(meteo, nieve, how='left')
    meteo = pd.merge(meteo, heladas, how='left')
    meteo = pd.merge(meteo, ventana_solana, how='left')
    meteo = pd.merge(meteo, ventana_umbria, how='left')
    meteo = meteo.set_index('estacion')
    meteo = meteo.fillna(0)
    meteo = meteo.convert_dtypes()
    meteo.loc['media'] = meteo.mean(axis=0)
    meteo.index.name = 'Periodo'
    meteo.columns=['días lluvia', '5 días sin lluvia', '8 días sin lluvia', 'nieve', 'heladas', 'ventana solana', 'ventana umbría']
    datos_meteo.append({'estacion': estacion, 'meteo':meteo})

# CERRAR CONEXIÓN BASE DE DATOS

conn.close()

# EXPORTAR 

capitulos = ['introduccion', 'objetivos', 'ambito', 'requisitos', 'puestos', 'planificacion', 'elementos', 'juicio']

env = Environment(loader=FileSystemLoader('static/templates_vera'))
template =env.get_template('base.html')
output = template.render(
    comarca = comarca,
    zona = zona,
    severidad = severidad,
    condiciones_sinopticas = incendios_cs.to_dict(orient='records'),
    ss = ss,
    catastro = catastro,
    forestales = forestales,
    capitulos = capitulos,
    rios = rios,
    datos_meteo = datos_meteo,
    causas = causas,
    causas_detalle = causas_detalle
)

with open('static/html/final.html', 'w') as f:
    f.write(output)

20

,idcausa,descripcion_corta,num_incendios,sup_arbolada,sup_no_arbolada,sup_total
0,100,Rayo,67.0,162.98,1428.65,1591.63
1,210,Quema agrícola (s.e.),68.0,16.19,75.95,92.14
2,211,Quema agrícola (rastrojos),9.0,0.69,1.43,2.12
3,212,Quema agrícola (restos poda),21.0,1.70,16.35,18.05
4,213,Quema agrícola (lindes y bordes fincas),6.0,0.00,17.53,17.53
5,220,Quema Reg. Pastos (s.e.),28.0,19.44,203.46,222.90
6,221,Quema Reg. Pastos (matorral),20.0,201.72,124.35,326.07
7,222,Quema Reg. Pastos (herbáceas),4.0,0.00,1.57,1.57
8,230,Trabajos forestales,29.0,11.22,78.62,89.84
9,240,Hogueras,4.0,0.00,2.78,2.78


,Número de incendios
Situación sinóptica,
Advención del Este,10
Advención del Noreste,1
Advención del Noroeste,4
Advención del Suroeste,1
Baja térmica,4


,gid,count,label,geom,idpif,year,fecha
0,2,591,1,"MULTIPOLYGON (((276458.553 4450098.592, 276463...",1163407,2018,2018-12-28
1,1,91,1,"MULTIPOLYGON (((278008.038 4449073.803, 278006...",1163407,2018,2018-12-28


{'zona': 'Losar',
 'idpif': ['1163407', '569396', '1169767', '574250', '1122672'],
 'incendios': [{'fecha': '28/12/2018', 'idpif': '1163407'}]}

...

,gid,count,label,geom,idpif,year,fecha
0,1,82,1,"MULTIPOLYGON (((279086.490 4451805.121, 279088...",574250,2017,2017-06-02


{'zona': 'Losar',
 'idpif': ['1163407', '569396', '1169767', '574250', '1122672'],
 'incendios': [{'fecha': '28/12/2018', 'idpif': '1163407'},
  {'fecha': '21/07/2016', 'idpif': '569396'},
  {'fecha': '16/04/2019', 'idpif': '1169767'},
  {'fecha': '02/06/2017', 'idpif': '574250'}]}

,gid,count,label,geom,idpif,year,fecha
0,1,168,1,"MULTIPOLYGON (((276439.269 4452792.611, 276441...",1122672,2013,2013-09-20


{'zona': 'Losar',
 'idpif': ['1163407', '569396', '1169767', '574250', '1122672'],
 'incendios': [{'fecha': '28/12/2018', 'idpif': '1163407'},
  {'fecha': '21/07/2016', 'idpif': '569396'},
  {'fecha': '16/04/2019', 'idpif': '1169767'},
  {'fecha': '02/06/2017', 'idpif': '574250'},
  {'fecha': '20/09/2013', 'idpif': '1122672'}]}

,forestal,contacto
0,Arturo Diaz Herrero,620452991
